In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, log_loss

In [3]:
import os
import sys

os.chdir(r"C:\Users\Jack Wright\Documents\GitHub\sevLandsPublicData\dynamic_win_probability")

In [4]:
df = pd.read_csv('../data/replay_data/transformed_replay_data_win_probability/dynamic_win_probability_50k.csv')

In [10]:
# print all the columsn in a sepearate window 
pd.set_option('display.max_columns', None)
print(df.head())

                            game_id  turn  on_play   won  user_hand_1  \
0  53401b113a4f425fa26e60edd314dd27     1     True  True        0.555   
1  53401b113a4f425fa26e60edd314dd27     2     True  True        0.559   
2  53401b113a4f425fa26e60edd314dd27     3     True  True          NaN   
3  53401b113a4f425fa26e60edd314dd27     4     True  True        0.555   
4  53401b113a4f425fa26e60edd314dd27     5     True  True        0.555   

   user_hand_2  user_hand_3  user_hand_4  user_hand_5  user_hand_6  \
0        0.541        0.551        0.558          NaN          NaN   
1        0.555        0.551        0.558          NaN          NaN   
2        0.559        0.555        0.551          NaN          NaN   
3        0.551          NaN          NaN          NaN          NaN   
4          NaN          NaN          NaN          NaN          NaN   

   user_hand_7  user_hand_8  user_hand_9  user_hand_10  user_hand_11  \
0          NaN          NaN          NaN           NaN           NaN

In [5]:
# data prep
data = df
X = data.drop(columns=['game_id', 'won'])
y = data['won']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [6]:
X_test

,turn,on_play,user_hand_1,user_hand_2,user_hand_3,user_hand_4,user_hand_5,user_hand_6,user_hand_7,user_hand_8,...,oppo_non_creatures_12,oppo_non_creatures_13,oppo_non_creatures_14,oppo_non_creatures_15,oppo_non_creatures_16,oppo_non_creatures_17,oppo_non_creatures_18,oppo_non_creatures_19,oppo_non_creatures_20,oppo_life
232464,1,False,NaN,0.548,NaN,0.566,0.512,NaN,0.554,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0
254210,1,False,NaN,NaN,NaN,0.551,0.535,0.540,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0
191303,6,False,0.502,0.542,0.541,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0
367977,1,True,NaN,NaN,0.557,0.525,0.547,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0
369097,2,True,0.541,0.542,0.529,0.528,NaN,0.535,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321311,10,True,0.525,0.548,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
404326,8,True,0.523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0
127915,8,True,0.562,0.530,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0
320961,11,False,0.539,0.507,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0


In [7]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, log_loss

# Simplified parameter grid
param_grid = {
    'n_estimators': [50, 100],      # Number of trees
    'learning_rate': [0.1],         # Fixed learning rate for simplicity
    'max_depth': [4, 6],            # Test a smaller range of tree depths
    'subsample': [0.8],             # Fixed subsample fraction
    'colsample_bytree': [0.8]       # Fixed feature fraction
}

# Initialize the model
xgb_model = XGBClassifier(eval_metric='logloss', use_label_encoder=False)

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='roc_auc',  # Use AUC as the evaluation metric
    cv=3,               # 3-fold cross-validation
    verbose=1,          # Show minimal progress during search
    n_jobs=-1           # Use all available CPU cores
)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Get the best model and its parameters
best_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

# Evaluate the best model
y_predict_proba = best_model.predict_proba(X_test)[:, 1]

# Calculate evaluation metrics
auc_score = roc_auc_score(y_test, y_predict_proba)
log_loss_score = log_loss(y_test, y_predict_proba)

print(f"AUC: {auc_score:.4f}")
print(f"Log Loss: {log_loss_score:.4f}")


Fitting 3 folds for each of 4 candidates, totalling 12 fits


c:\ProgramData\miniforge3\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:01:40] WARNING: D:\bld\xgboost-split_1733179535861\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'subsample': 0.8}
AUC: 0.7586
Log Loss: 0.5747


In [8]:
import pickle

# Save the trained model to a file
# model_filename = '../models/win_probability/xgb_win_prob_model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(best_model, file)

print(f"Model saved to {model_filename}")


Model saved to ../models/win_probability/xgb_win_prob_model.pkl
